# Create `asset-data.json` file for Mean-Variance Analyzer

In this notebook, we get and clean the financial market data that will be preloaded as `asset-data.json` in the "Mean-Variance Analyzer" web app ([view live site](https://mvanalyzer.dev/)).

In [1]:
import pandas as pd
import yfinance as yf
import json

## Getting monthly adjusted close % return for assets

We start by getting the financial market data via the yfinance API. We then sort it in alphabetical order - this allows us to store the information needed to recreate the covariance matrix for these assets on the client's device in a file approximately half the size (since the matrix is symmetric - please see the site's [Background](https://mvanalyzer.dev/background/) page for more information on the covariance matrix and how other quantities contained in this dataset are used in the app). We have chosen 189 popular assets for our demo data including stocks, ETFs, cryptocurrencies and more.

In [2]:
assetTickers = ['005930.KS', '2222.SR', 'AAPL', 'ABBV', 'ABNB', 'ABT', 'ADA-USD', 'ADBE', 'ADI', 'ADP', 'ADSK', 'AEP', 
                'AJRD', 'AMAGX', 'AMANX', 'AMAT', 'AMD', 'AMGN', 'AMZN', 'APO', 'AR', 'ASML', 'ASTS', 'ATVI', 'AVGO', 
                'AXP', 'B', 'BA', 'BABA', 'BAC', 'BHVN', 'BJ', 'BKNG', 'BLK', 'BMY', 'BNB-USD', 'BNP.PA', 'BRK-B', 
                'BTC-USD', 'BX', 'C', 'CAR', 'CAT', 'CDNS', 'CHK', 'CHTR', 'CL=F', 'CMCSA', 'COIN', 'COST', 'CRM', 
                'CSCO', 'CSX', 'CTAS', 'CVS', 'CVX', 'DBX', 'DELL', 'DIS', 'DOGE-USD', 'DOT-USD', 'DOW', 'DXCM', 
                'EBAY', 'ETC-USD', 'ETH-USD', 'EURUSD=X', 'FIS', 'FISV', 'FTNT', 'GBPUSD=X', 'GC=F', 'GD', 'GE', 
                'GILD', 'GLD', 'GOOG', 'GOOGL', 'GS', 'GSAT', 'HD', 'HLAL', 'HON', 'HOOD', 'HPQ', 'HWM', 'IBM', 'IBN', 
                'INTC', 'INTU', 'IVZ', 'JD', 'JNJ', 'JOBY', 'JPM', 'JPY=X', 'KAMN', 'KKR', 'KO', 'LLY', 
                'LTC-USD', 'LYFT', 'MA', 'MATIC-USD', 'MC.PA', 'MCD', 'META', 'MMM', 'MRK', 'MS', 'MSFT', 'MUFG', 
                'NESN.SW', 'NFLX', 'NKE', 'NOC', 'NOK', 'NU', 'NVDA', 'NVO', 'ORCL', 'OVV', 'PANW', 'PDD', 'PEP', 
                'PFE', 'PFGC', 'PG', 'PL', 'PNC', 'PYPL', 'QCOM', 'RKLB', 'ROG.SW', 'RTX', 'SAP', 'SCHW', 'SCU', 
                'SHOP', 'SI=F', 'SOFI', 'SOL-USD', 'SONY', 'SPCE', 'SPGI', 'SPR', 'SPUS', 'SPY', 'SQ', 'SWN', 'TCEHY', 
                'TDG', 'TEAM', 'TGI', 'TMO', 'TRV', 'TRX-USD', 'TSLA', 'TSM', 'TTEK', 'TXN', 'UBER', 'UNH', 'USB', 
                'V', 'VEA', 'VMW', 'VORB', 'VTI', 'VTOL', 'VZ', 'WBA', 'WFC', 'WIZEY', 'WMT', 'WSC', 'XLM-USD', 
                'XMR-USD', 'XOM', 'XRP-USD', '^CMC200', '^DJI', '^FTSE', '^GSPC', '^IXIC', '^N225', '^RUT', '^TNX', 
                '^TYX']

assetTickers.sort()

In [3]:
print(len(assetTickers)) # Check that it contains 189 assets
print(len(set(assetTickers))) # Check that it contains 189 unique assets

189
189


Now that we have the tickers for our assets of interest, let us make a pandas DataFrame of the monthly % return of the adjusted close price over the max period of our assets' data as provided by the yfinance API. We are cleaning the data by first forward filling all NaN values for close prices and then dropping any dates that are before ALL assets have data. This is one of the multiple choices in this document made by the developer that will inevitably affect the accuracy of the results - note that we are not liable for the accuracy of this data nor its resulting information as per the site's [Terms of Service](https://mvanalyzer.dev/terms/). This data is also only up to January 2023 and will be stale after that - it is only meant for educational demonstration and not as financial advice.

In [4]:
%%time
df = pd.DataFrame()
for ticker in assetTickers:
    # Get monthly max period close data
    try: # Workaround for yfinance version 0.2.9 - opened issue #1405 on GitHub
        tmpDf = pd.DataFrame(yf.Ticker(ticker).history(period="max", interval="1mo")["Close"]).rename(
            columns={"Close":ticker})
    except:
        tmpDf = pd.DataFrame(yf.Ticker(ticker).history(period="max", interval="1wk")["Close"]).rename(
            columns={"Close":ticker})
    
    # Format for monthly index using the last datum of each month
    tmpDf.index = tmpDf.index.strftime('%Y-%m')
    tmpDf = tmpDf[~tmpDf.index.duplicated(keep='last')]
    
    df = df.join(tmpDf, how='outer')

# clean data
df.fillna(method='ffill', inplace=True)
df.dropna(how='all', inplace=True)
df.drop(df.index[-1], inplace=True) # delete data from last month that hasn't finished yet

# get monthly pct return
df = df.pct_change()[1:] * 100

Wall time: 2min 49s


In [5]:
df.head()

,005930.KS,2222.SR,AAPL,ABBV,ABNB,ABT,ADA-USD,ADBE,ADI,ADP,...,XRP-USD,^CMC200,^DJI,^FTSE,^GSPC,^IXIC,^N225,^RUT,^TNX,^TYX
Date,,,,,,,,,,,,,,,,,,,,,
1962-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1962-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1962-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1962-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1962-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
df["BA"].iloc[:5]

Date
1962-02   -4.328018
1962-03   -4.523957
1962-04   -6.234279
1962-05   -6.457078
1962-06   -6.321804
Name: BA, dtype: float64

In [7]:
df.iloc[-5:]

,005930.KS,2222.SR,AAPL,ABBV,ABNB,ABT,ADA-USD,ADBE,ADI,ADP,...,XRP-USD,^CMC200,^DJI,^FTSE,^GSPC,^IXIC,^N225,^RUT,^TNX,^TYX
Date,,,,,,,,,,,,,,,,,,,,,
2022-09,-11.055275,-3.613675,-11.975600,1.496924,-7.142859,-5.600001,-2.742769,-26.306767,-5.799215,-4.743626,...,46.330552,-7.075179,-8.838089,-5.359551,-9.339567,-10.498976,-7.668925,-9.728219,21.417173,15.668202
2022-10,12.633037,-3.068329,10.955140,6.918496,1.780277,1.832092,-6.463417,15.734006,3.552457,4.717272,...,-2.888653,9.210288,13.950804,2.911315,7.986341,3.900767,6.362481,10.941180,7.176663,11.660023
2022-11,4.713803,-4.460432,-3.462885,11.980885,-4.461701,10.217186,-21.522338,8.298271,16.868812,13.949168,...,-12.270908,-15.717265,5.672634,6.746072,5.375289,4.366973,1.382981,2.150676,-9.173413,-9.086583
2022-12,-11.093244,-2.419095,-12.081657,-1.421261,-16.291364,1.572770,-22.709585,-2.435275,-2.294836,-11.071733,...,-16.835513,-6.312361,-4.170365,-1.603041,-5.897147,-8.733166,-6.702030,-6.643236,4.752900,4.003136
2023-01,11.724934,2.647982,11.052106,-9.412423,29.953217,2.282679,58.612551,10.046652,9.016649,-5.672780,...,19.551136,37.168548,2.832178,4.294322,6.175286,10.682381,4.723637,9.691409,-9.022942,-7.899369


In [8]:
df.iloc[-1].isna().sum() # Make sure all assets have data up to Jan 2023

0

## Extracting the mean-variance analysis data

For the basic mean-variance analysis used by the app, we want to extract from the data each tickers' long name, annualized mean monthly % return, the variance of these returns, and the covariances of these returns with respect to those of each of the other assets. To calculate the covariances between any two of the assets, the developer first chose to find the earliest data both assets had valid (non-NaN) data and only calculate the covariance over the periods where both assets have data, with the intention that this may better capture the diversification that mean-variance analysis aims for (since the variance and average of each asset is calculated over its max period). Note that this affects the accuracy of the data and may not be part of the standard calculation of the ex post Sharpe ratio.

In [9]:
firstValidMonthDict = dict()
for ticker in assetTickers:
    firstValidMonthDict[ticker] = df[ticker].notna().idxmax()

Now we use the DataFrame of historical monthly returns to create a dictionary containing the annualized mean return and variance of each asset over its max period and the covariance between each asset and each other asset that comes later than it in alphabetical order over the periods where they both have valid data. 

In [10]:
# Workaround for yfinance version 0.2.9 - open issue #1407 on GitHub
assetTitles = {'005930.KS': 'Samsung Electronics Co., Ltd.', '2222.SR': 'Saudi Arabian Oil Company', 
               'AAPL': 'Apple Inc.', 'ABBV': 'AbbVie Inc.', 'ABNB': 'Airbnb, Inc.', 'ABT': 'Abbott Laboratories',
              'ADA-USD': 'Cardano USD', 'ADBE': 'Adobe Inc.', 'ADI': 'Analog Devices, Inc.', 
               'ADP': 'Automatic Data Processing, Inc.', 'ADSK': 'Autodesk, Inc.', 
               'AEP': 'American Electric Power Company, Inc.', 'AJRD': 'Aerojet Rocketdyne Holdings, Inc.',
              'AMAGX': 'Amana Mutual Funds Trust Growth Fund', 'AMANX': 'Amana Mutual Funds Trust Income Fund',
              'AMAT': 'Applied Materials, Inc.', 'AMD': 'Advanced Micro Devices, Inc.', 'AMGN': 'Amgen Inc.',
              'AMZN': 'Amazon.com, Inc.', 'APO': 'Apollo Global Management, Inc.', 
               'AR': 'Antero Resources Corporation', 'ASML': 'ASML Holding N.V.', 'ASTS': 'AST SpaceMobile, Inc.',
              'ATVI': 'Activision Blizzard, Inc.', 'AVGO': 'Broadcom Inc.', 'AXP': 'American Express Company',
              'B': 'Barnes Group Inc.', 'BA': 'The Boeing Company', 'BABA': 'Alibaba Group Holding Limited',
              'BAC': 'Bank of America Corporation', 'BHVN': 'Biohaven Ltd.', 
               'BJ': "BJ's Wholesale Club Holdings, Inc.", 'BKNG': 'Booking Holdings Inc.', 'BLK': 'BlackRock, Inc.',
              'BMY': 'Bristol-Myers Squibb Company', 'BNB-USD': 'BNB USD', 'BNP.PA': 'BNP Paribas SA',
              'BRK-B': 'Berkshire Hathaway Inc.', 'BTC-USD': 'Bitcoin USD', 'BX': 'Blackstone Inc.',
              'C': 'Citigroup Inc.', 'CAR': 'Avis Budget Group, Inc.', 'CAT': 'Caterpillar Inc.',
              'CDNS': 'Cadence Design Systems, Inc.', 'CHK': 'Chesapeake Energy Corporation',
              'CHTR': 'Charter Communications, Inc.', 'CL=F': 'Crude Oil Mar 23', 'CMCSA': 'Comcast Corporation',
              'COIN': 'Coinbase Global, Inc.', 'COST': 'Costco Wholesale Corporation', 'CRM': 'Salesforce, Inc.',
              'CSCO': 'Cisco Systems, Inc.', 'CSX': 'CSX Corporation', 'CTAS': 'Cintas Corporation',
              'CVS': 'CVS Health Corporation', 'CVX': 'Chevron Corporation', 'DBX': 'Dropbox, Inc.',
              'DELL': 'Dell Technologies Inc.', 'DIS': 'The Walt Disney Company', 'DOGE-USD': 'Dogecoin USD',
              'DOT-USD': 'Polkadot USD', 'DOW': 'Dow Inc.', 'DXCM': 'DexCom, Inc.', 'EBAY': 'eBay Inc.',
              'ETC-USD': 'Ethereum Classic USD', 'ETH-USD': 'Ethereum USD', 'EURUSD=X': 'EUR/USD', 
               'FIS': 'Fidelity National Information Services, Inc.', 'FISV': 'Fiserv, Inc.', 'FTNT': 'Fortinet, Inc.',
              'GBPUSD=X': 'USD/GBP', 'GC=F': 'Gold Apr 23', 'GD': 'General Dynamics Corporation', 
              'GE': 'General Electric Company', 'GILD': 'Gilead Sciences, Inc.', 'GLD': 'SPDR Gold Shares',
              'GOOG': 'Alphabet Inc.', 'GOOGL': 'Alphabet Inc.', 'GS': 'The Goldman Sachs Group, Inc.',
              'GSAT': 'Globalstar, Inc.', 'HD': 'The Home Depot, Inc.', 'HLAL': 'Wahed FTSE USA Shariah ETF',
              'HON': 'Honeywell International Inc.', 'HOOD': 'Robinhood Markets, Inc.', 'HPQ': 'HP Inc.',
              'HWM': 'Howmet Aerospace Inc.', 'IBM': 'International Business Machines Corporation',
              'IBN': 'ICICI Bank Limited', 'INTC': 'Intel Corporation', 'INTU': 'Intuit Inc.', 'IVZ': 'Invesco Ltd.',
              'JD': 'JD.com, Inc.', 'JNJ': 'Johnson & Johnson', 'JOBY': 'Joby Aviation, Inc.', 
               'JPM': 'JPMorgan Chase & Co.', 'JPY=X': 'USD/JPY', 'KAMN': 'Kaman Corporation', 'KKR': 'KKR & Co. Inc.',
              'KO': 'The Coca-Cola Company', 'LLY': 'Eli Lilly and Company', 'LTC-USD': 'Litecoin USD',
              'LYFT': 'Lyft, Inc.', 'MA': 'Mastercard Incorporated', 'MATIC-USD': 'Polygon USD',
              'MC.PA': 'LVMH Moët Hennessy - Louis Vuitton, Société Européenne', 'MCD': "McDonald's Corporation",
              'META': 'Meta Platforms, Inc.', 'MMM': '3M Company', 'MRK': 'Merck & Co., Inc.', 'MS': 'Morgan Stanley',
              'MSFT': 'Microsoft Corporation', 'MUFG': 'Mitsubishi UFJ Financial Group, Inc.', 
               'NESN.SW': 'Nestlé S.A.', 'NFLX': 'Netflix, Inc.', 'NKE': 'NIKE, Inc.', 
               'NOC': 'Northrop Grumman Corporation', 'NOK': 'Nokia Oyj', 'NU': 'Nu Holdings Ltd.', 
               'NVDA': 'NVIDIA Corporation', 'NVO': 'Novo Nordisk A/S', 'ORCL': 'Oracle Corporation', 
               'OVV': 'Ovintiv Inc.', 'PANW': 'Palo Alto Networks, Inc.', 'PDD': 'Pinduoduo Inc.', 
               'PEP': 'PepsiCo, Inc.', 'PFE': 'Pfizer Inc.', 'PFGC': 'Performance Food Group Company',
              'PG': 'The Procter & Gamble Company', 'PL': 'Platinum Apr 23', 
               'PNC': 'The PNC Financial Services Group, Inc.', 'PYPL': 'PayPal Holdings, Inc.', 
              'QCOM': 'QUALCOMM Incorporated', 'RKLB': 'Rocket Lab USA, Inc.', 'ROG.SW': 'Roche Holding AG',
              'RTX': 'Raytheon Technologies Corporation', 'SAP': 'SAP SE', 'SCHW': 'The Charles Schwab Corporation',
              'SCU': 'Sculptor Capital Management, Inc.', 'SHOP': 'Shopify Inc.', 'SI=F': 'Silver Mar 23',
              'SOFI': 'SoFi Technologies, Inc.', 'SOL-USD': 'Solana USD', 'SONY': 'Sony Group Corporation',
              'SPCE': 'Virgin Galactic Holdings, Inc.', 'SPGI': 'S&P Global Inc.', 
               'SPR': 'Spirit AeroSystems Holdings, Inc.', 'SPUS': 'SP Funds S&P 500 Sharia Industry Exclusions ETF',
              'SPY': 'SPDR S&P 500 ETF Trust', 'SQ': 'Block, Inc.', 'SWN': 'Southwestern Energy Company',
              'TCEHY': 'Tencent Holdings Limited', 'TDG': 'TransDigm Group Incorporated', 
               'TEAM': 'Atlassian Corporation', 'TGI': 'Triumph Group, Inc.', 'TMO': 'Thermo Fisher Scientific Inc.',
              'TRV': 'The Travelers Companies, Inc.', 'TRX-USD': 'TRON USD', 'TSLA': 'Tesla, Inc.', 
               'TSM': 'Taiwan Semiconductor Manufacturing Company Limited', 'TTEK': 'Tetra Tech, Inc.',
              'TXN': 'Texas Instruments Incorporated', 'UBER': 'Uber Technologies, Inc.', 
               'UNH': 'UnitedHealth Group Incorporated', 'USB': 'U.S. Bancorp', 'V': 'Visa Inc.', 
               'VEA': 'Vanguard Developed Markets Index Fund', 'VMW': 'VMware, Inc.', 
               'VORB': 'Virgin Orbit Holdings, Inc.', 'VTI': 'Vanguard Total Stock Market Index Fund',
              'VTOL': 'Bristow Group Inc.', 'VZ': 'Verizon Communications Inc.', 
               'WBA': 'Walgreens Boots Alliance, Inc.', 'WFC': 'Wells Fargo & Company', 'WIZEY': 'Wise plc',
              'WMT': 'Walmart Inc.', 'WSC': 'WillScot Mobile Mini Holdings Corp.', 'XLM-USD': 'Stellar USD',
              'XMR-USD': 'Monero USD', 'XOM': 'Exxon Mobil Corporation', 'XRP-USD': 'XRP USD', 
               '^CMC200': 'CMC Crypto 200 Index by Solacti', '^DJI': 'Dow Jones Industrial Average', 
               '^FTSE': 'FTSE 100', '^GSPC': 'S&P 500', '^IXIC': 'NASDAQ Composite', '^N225': 'Nikkei 225',
              '^RUT': 'Russell 2000', '^TNX': 'Treasury Yield 10 Years', '^TYX': 'Treasury Yield 30 Years'}

In [11]:
%%time
assetDataDict = dict()
for i in range(len(assetTickers)-1): # Calculate covariances for all except last asset (alphabetical order)
    assetVar = 12 * (df[assetTickers[i]].std()**2) # Annualized %: multiply by 12 months/year
    assetMean = 12 * df[assetTickers[i]].mean()
    
    try: # Workaround for yfinance version 0.2.9 - open issue #1407 on GitHub
        assetInfo = yf.Ticker(assetTickers[i]).info
        if 'longName' in assetInfo and assetInfo['longName']:
            assetTitle = assetInfo['longName']
        else:
            assetTitle = assetInfo['shortName']
    except:
        assetTitle = assetTitles[assetTickers[i]]
        
    assetDataDict[assetTickers[i]] = {'title': assetTitle, 'annRetPct': assetMean, 
                                      'annVar': assetVar, 'cov': {}}
    
    for j in range(i+1, len(assetTickers)):
        startDate = max(firstValidMonthDict[assetTickers[i]], firstValidMonthDict[assetTickers[j]])
        assetCov = df[[assetTickers[i], assetTickers[j]]].loc[startDate:].cov().iloc[0, 1]
        assetDataDict[assetTickers[i]]['cov'][assetTickers[j]] = 12 * assetCov

# Calculate values for last asset (alphabetical order)
assetVar = 12 * (df[assetTickers[-1]].std()**2)
assetMean = 12 * df[assetTickers[-1]].mean()

try: # Manual workaround for yfinance version 0.2.9 - open issue #1407 on GitHub
    assetInfo = yf.Ticker(assetTickers[-1]).info
    if 'longName' in assetInfo and assetInfo['longName']:
        assetTitle = assetInfo['longName']
    else:
        assetTitle = assetInfo['shortName']
except:
    assetTitle = assetTitles[assetTickers[-1]]
        
assetDataDict[assetTickers[-1]] = {'title': assetTitle, 'annRetPct': assetMean, 
                                  'annVar': assetVar}

Wall time: 8min 34s


In [12]:
print(assetDataDict['^TNX'])
print(assetDataDict['^TYX'])

{'title': 'Treasury Yield 10 Years', 'annRetPct': 1.0096391435623997, 'annVar': 808.4226508725678, 'cov': {'^TYX': 551.4673170243406}}
{'title': 'Treasury Yield 30 Years', 'annRetPct': -0.8042711734499879, 'annVar': 428.32914766334875}


## Export dictionary as JSON file

Now we can export the dictionary with the data required by the [Mean-Variance Analyzer](https://mvanalyzer.dev/) web app as a JSON file to its data folder.

In [13]:
with open("../mean-variance-analyzer/data/asset-data.json", "w") as f:
    json.dump(assetDataDict, f, indent=2)